## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [0]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [0]:
def get_parallel_dbs(db):
  dbs=list()
  for i in range(len(db)):
    dbs.append(torch.cat((db[:i],db[i+1:])))
  return dbs

In [0]:
dbs=get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(num_entries):
  db = torch.rand(num_entries) > 0.5
  dbs=get_parallel_dbs(db)
  return db,dbs



In [0]:
def get_parallel_db(db,index):
  dbs=torch.cat((db[:index],db[index+1:]))
  return dbs

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [0]:
db, pdbs = create_db_and_parallels(20)

In [0]:
def query(db):
    return db.sum()

In [0]:
full_db_result = query(db)
print(full_db_result)

NameError: ignored

In [0]:
sensitivity = 0

for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [0]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [0]:
# try this project here!

In [0]:
def sensitivity(func,num_entries):
  db = torch.rand(num_entries) > 0.5
  dbs=get_parallel_dbs(db)
  result_db=func(db)
  sensitivity = 0
  for pdb in dbs:
    pdb_result = func(pdb)
    
    db_distance = torch.abs(pdb_result - result_db)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance
        
  return sensitivity
        

In [0]:
sen=sensitivity(mean,1000)

In [0]:
sen

tensor(0.0005)

In [0]:
 def mean(a):
    return a.float().mean()

In [0]:
mean(db)-mean(dbs[0])

tensor(0.0981)

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [0]:
def threshold(db,th=5):
  return (db.sum()>th).float()

In [0]:
for i in range(10):
  sens_f=sensitivity(query,10)
  print(sens_f)

tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [0]:
db,_=create_db_and_parallels(100)

In [0]:
db.sum()
db_without=torch.cat((db[0:10],db[11:]))
db_without.sum()


tensor(48)

In [0]:
db.sum()-db_without.sum()


tensor(1)

In [0]:
db.float().sum()/len(db)-db_without.float().sum()/len(db_without)


tensor(0.0052)

In [0]:
(sum(db).float()>48)-(sum(db_without).float()>48)

tensor(1, dtype=torch.uint8)

tensor(49)

tensor(49)

100

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [0]:
coinone=torch.rand(1)>0.5
cointwo=torch.rand(1)>0.5
print(coinone)


tensor([1], dtype=torch.uint8)


In [0]:
db_one,_=create_db_and_parallels(10)
db_two,_=create_db_and_parallels(100)
db_three,_=create_db_and_parallels(1000)
db_four,_=create_db_and_parallels(10000)


In [0]:
def db_noise(entry):
  db,_=create_db_and_parallels(entry)
  dbn=torch.zeros(entry)
  for i in range(len(db)):
    coinone=torch.rand(1)>0.5
    cointwo=torch.rand(1)>0.5
    if coinone==1:
      dbn[i]=db[i]
    else:
      dbn[i]=cointwo 
  return db,dbn

db_one,db_one_noise=db_noise(10)
print("simple: {} , noise: {} ".format(db_one.float().mean(),db_one_noise.float().mean()))
db_two,db_two_noise=db_noise(100)
print("simple: {} , noise: {} ".format(db_two.float().mean(),db_two_noise.float().mean()))
db_three,db_three_noise=db_noise(1000)
print("simple: {} , noise: {} ".format(db_three.float().mean(),db_three_noise.float().mean()))
db_four,db_four_noise=db_noise(10000)
print("simple: {} , noise: {} ".format(db_four.float().mean(),db_four_noise.float().mean()))

simple: 0.6000000238418579 , noise: 0.30000001192092896 
simple: 0.5299999713897705 , noise: 0.5099999904632568 
simple: 0.4950000047683716 , noise: 0.49799999594688416 
simple: 0.5019999742507935 , noise: 0.5019999742507935 


In [0]:
def db_noise(entry):
  db,_=create_db_and_parallels(entry)

  coinone=(torch.rand(entry)>0.5).float()
  cointwo=(torch.rand(entry)>0.5).float()
  dbn=db.float()*coinone+(1-coinone)*cointwo
  dbs=((torch.mean(dbn.float()))*2)-0.5
  return db,dbn,dbs

db_one,db_one_noise,dbs_one=db_noise(10)
print("simple: {} , noise: {} , deskewed: {}".format(db_one.float().mean(),db_one_noise.float().mean(),dbs_one))
db_two,db_two_noise,dbs_two=db_noise(100)
print("simple: {} , noise: {} , deskewed: {}".format(db_two.float().mean(),db_two_noise.float().mean(),dbs_two))
db_three,db_three_noise,dbs_three=db_noise(1000)
print("simple: {} , noise: {} , deskewed: {}".format(db_three.float().mean(),db_three_noise.float().mean(),dbs_three))
db_four,db_four_noise,dbs_four=db_noise(10000)
print("simple: {} , noise: {} , deskewed: {}".format(db_four.float().mean(),db_four_noise.float().mean(),dbs_four))

simple: 0.4000000059604645 , noise: 0.4000000059604645 , deskewed: 0.30000001192092896
simple: 0.5799999833106995 , noise: 0.5400000214576721 , deskewed: 0.5800000429153442
simple: 0.5350000262260437 , noise: 0.5220000147819519 , deskewed: 0.5440000295639038
simple: 0.5016000270843506 , noise: 0.49970000982284546 , deskewed: 0.4994000196456909


In [0]:
(mean(db_one_noise)-0.25)*2

tensor(0.1000)

# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [0]:
noise=0.1
def db_noise(entry,noise):
  db,_=create_db_and_parallels(entry)

  coinone=(torch.rand(entry)>noise).float()
  cointwo=(torch.rand(entry)>0.5).float()
  dbn=db.float()*coinone+(1-coinone)*cointwo
  dbs=((torch.mean(dbn.float()))-(noise*0.5))/(1-noise)
  return db,dbn,dbs

db_one,db_one_noise,dbs_one=db_noise(10,noise)
print("simple: {} , noise: {} , deskewed: {}".format(db_one.float().mean(),db_one_noise.float().mean(),dbs_one))
db_two,db_two_noise,dbs_two=db_noise(100,noise)
print("simple: {} , noise: {} , deskewed: {}".format(db_two.float().mean(),db_two_noise.float().mean(),dbs_two))
db_three,db_three_noise,dbs_three=db_noise(1000,noise)
print("simple: {} , noise: {} , deskewed: {}".format(db_three.float().mean(),db_three_noise.float().mean(),dbs_three))
db_four,db_four_noise,dbs_four=db_noise(10000,noise)
print("simple: {} , noise: {} , deskewed: {}".format(db_four.float().mean(),db_four_noise.float().mean(),dbs_four))

simple: 0.20000000298023224 , noise: 0.20000000298023224 , deskewed: 0.1666666716337204
simple: 0.41999998688697815 , noise: 0.41999998688697815 , deskewed: 0.41111108660697937
simple: 0.49000000953674316 , noise: 0.47600001096725464 , deskewed: 0.47333335876464844
simple: 0.49950000643730164 , noise: 0.5013999938964844 , deskewed: 0.5015555620193481


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [0]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(40.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [0]:
import torch

def get_parallel_dbs(db):
  dbs=list()
  for i in range(len(db)):
    dbs.append(torch.cat((db[:i],db[i+1:])))
  return dbs
def create_db_and_parallels(num_entries):
  db = torch.rand(num_entries) > 0.5
  dbs=get_parallel_dbs(db)
  return db,dbs

def query(db):
    return db.sum()

def mean(a):
   return a.float().mean()
  

In [0]:
epsilon=0.7

import numpy as np

def sensitivity_two(func,db,dbs):
  result_db=func(db)
  sensitivity = 0
  
  for pdb in dbs:
    
    pdb_result = func(pdb)
    
    db_distance = torch.abs(pdb_result - result_db)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance
        
  return sensitivity

In [0]:
        
def laplacian_mechanism(entry,query):
  db,dbs=create_db_and_parallels(entry)
  sen=sensitivity_two(query,db,dbs)
  beta=sen/torch.tensor(epsilon)
  n1=np.random.laplace(0,beta,1)
  noise=torch.tensor(n1)
  return query(db),(query(db)+noise)




In [0]:
db_one,db_one_noise=laplacian_mechanism(10,mean)
print("simple: {} , noise: {} ".format(db_one,db_one_noise))
db_two,db_two_noise=laplacian_mechanism(100,mean)
print("simple: {} , noise: {} ".format(db_two,db_two_noise))
db_three,db_three_noise=laplacian_mechanism(1000,mean)
print("simple: {} , noise: {} ".format(db_three,db_three_noise))
db_four,db_four_noise=laplacian_mechanism(10000,mean)
print("simple: {} , noise: {} ".format(db_four,db_four_noise))


simple: 0.5 , noise: tensor([0.5922], dtype=torch.float64) 
simple: 0.5 , noise: tensor([0.4975], dtype=torch.float64) 
simple: 0.5320000052452087 , noise: tensor([0.5337], dtype=torch.float64) 
simple: 0.5026000142097473 , noise: tensor([0.5027], dtype=torch.float64) 


tensor(1)


simple: 5 , noise: tensor([5.5908], dtype=torch.float64) 
simple: 51 , noise: tensor([54.5943], dtype=torch.float64) 
simple: 500 , noise: tensor([498.8116], dtype=torch.float64) 
simple: 5037 , noise: tensor([5042.9325], dtype=torch.float64) 


# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [0]:
import numpy as np 

In [0]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [0]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions


In [0]:
preds.shape

(10000, 10)

In [0]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [0]:
new_labels

# PATE Analysis

In [0]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [0]:
from syft.frameworks.torch.differential_privacy import pate

In [0]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers
preds.shape
preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps
preds.shape


In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [0]:
preds[:,0:50] *= 0

array([6, 5, 4, 9, 3, 7, 9, 0, 5, 3, 7, 2, 7, 0, 0, 1, 2, 8, 8, 0, 9, 6,
       8, 8, 4, 9, 3, 7, 8, 5, 2, 2, 1, 0, 2, 4, 4, 0, 2, 7, 7, 8, 6, 9,
       7, 4, 5, 2, 9, 6, 8, 0, 3, 6, 5, 5, 6, 9, 3, 3, 4, 0, 4, 3, 6, 9,
       2, 1, 2, 1, 6, 7, 6, 3, 0, 8, 6, 8, 5, 3, 8, 1, 8, 5, 9, 6, 1, 5,
       7, 6, 0, 4, 4, 0, 2, 7, 3, 3, 5, 5])

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [0]:
import torch 
import torch.nn as nn
import torch.nn.functional as f
from torch import optim
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


0it [00:00, ?it/s]

9920512it [00:01, 9718407.41it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 142758.71it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2497110.84it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51315.90it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [0]:
test_data = mnist_trainset.test_data
test_targets = mnist_trainset.test_labels

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
# train_data_one=train_data[:10000,:]
# print(train_data_one.shape)
# train_targets_one=train_targets[:10000]
# print(train_targets_one.shape)
# train_data_two=train_data[10000:20000,:]
# print(train_data_two.shape)
# train_targets_two=train_targets[10000:20000]
# print(train_targets_two.shape)
# train_data_three=train_data[20000:30000,:]
# print(train_data_three.shape)
# train_targets_three=train_targets[20000:30000]
# print(train_targets_three.shape)
# train_data_four=train_data[30000:40000,:]
# print(train_data_four.shape)
# train_targets_four=train_targets[30000:40000]
# print(train_targets_four.shape)
# train_data_five=train_data[40000:50000,:]
# print(train_data_five.shape)
# train_targets_five=train_targets[40000:50000]
# print(train_targets_five.shape)
# train_data_six=train_data[50000:,:]
# print(train_data_six.shape)
# train_targets_six=train_targets[50000:]
# print(train_targets_six.shape)



torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([10000, 28, 28])
torch.Size([10000])


In [0]:

from torch.utils.data import TensorDataset,DataLoader
batch_size=100
def data_label(data,targets,size):
  dataset=TensorDataset(data,targets)
  data=DataLoader(dataset,batch_size=size,shuffle=True)
  return data
def data_label_test(data,targets,size):
  dataset=TensorDataset(data,targets)
  data=DataLoader(dataset,batch_size=size,shuffle=False)
  return data

In [0]:
class digit_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1=nn.Linear(784,256)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,100)
        self.fc4=nn.Linear(100,10)
        
        self.dropout=nn.Dropout(p=0.2)
        
    def forward(self,x):
        x = x.view(x.shape[0], -1)
        x=self.dropout(f.relu(self.fc1(x)))
        x=self.dropout(f.relu(self.fc2(x)))
        x=self.dropout(f.relu(self.fc3(x)))
        
        x=f.log_softmax(self.fc4(x),dim=1)
        return x
      

In [0]:
size=100
teacher=list()
student=list()
testloader=data_label_test(test_data,test_targets,size)


In [0]:
teacher.clear()
for i in range(6):
  model=digit_classifier()
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.003)
  epochs = 30
  steps = 0
  train_losses, test_losses = [], []
  size=100
  data=train_data[i*10000:((i+1)*10000),:]
  label=train_targets[i*10000:((i+1)*10000)]
  trainloader=data_label(data,label,size)
  for e in range(epochs):
      running_loss = 0
      for images, labels in trainloader:
          images=images.float()
          optimizer.zero_grad()

          log_ps = model(images)
          loss = criterion(log_ps, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
      else:
          test_loss=0
          accuracy=0
          with torch.no_grad():
              model.eval()
          # validation pass here
              for images, labels in testloader:
                  images=images.float()
                  val=model(images)
                  test_loss += criterion(val, labels)
                  val_prob= torch.exp(val)
                  top_p, top_class = val_prob.topk(1, dim=1)
                  
                  equals = top_class == labels.view(*top_class.shape)
                  accuracy += torch.mean(equals.type(torch.FloatTensor))
          model.train()
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(testloader))
          print("Epoch:{}/{}.. ".format(e+1, epochs),
               "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
               "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
               "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
  else:
    with torch.no_grad():
              model.eval()
          # validation pass here
              for images, labels in testloader:
                  images=images.float()
                  val=model(images)
                  val_prob= torch.exp(val)
                  top_p, top_class = val_prob.topk(1, dim=1)
                  student.append(top_class)
              teacher.append(list(student))
              student.clear()
    model.train()
    

Epoch:1/30..  Training Loss: 1.116..  Test Loss: 0.390..  Test Accuracy: 0.887
Epoch:2/30..  Training Loss: 0.426..  Test Loss: 0.272..  Test Accuracy: 0.924
Epoch:3/30..  Training Loss: 0.346..  Test Loss: 0.297..  Test Accuracy: 0.915
Epoch:4/30..  Training Loss: 0.301..  Test Loss: 0.254..  Test Accuracy: 0.929
Epoch:5/30..  Training Loss: 0.277..  Test Loss: 0.271..  Test Accuracy: 0.923
Epoch:6/30..  Training Loss: 0.272..  Test Loss: 0.244..  Test Accuracy: 0.936
Epoch:7/30..  Training Loss: 0.256..  Test Loss: 0.295..  Test Accuracy: 0.926
Epoch:8/30..  Training Loss: 0.256..  Test Loss: 0.230..  Test Accuracy: 0.943
Epoch:9/30..  Training Loss: 0.224..  Test Loss: 0.231..  Test Accuracy: 0.944
Epoch:10/30..  Training Loss: 0.207..  Test Loss: 0.269..  Test Accuracy: 0.932
Epoch:11/30..  Training Loss: 0.222..  Test Loss: 0.269..  Test Accuracy: 0.941
Epoch:12/30..  Training Loss: 0.229..  Test Loss: 0.277..  Test Accuracy: 0.935
Epoch:13/30..  Training Loss: 0.203..  Test Loss:

KeyboardInterrupt: ignored

In [0]:
teacher.clear()
for i in range(120):
  model=digit_classifier()
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.003)
  epochs = 30
  steps = 0
  train_losses, test_losses = [], []
  size=100
  data=train_data[i*500:((i+1)*500),:]
  label=train_targets[i*500:((i+1)*500)]
  trainloader=data_label(data,label,size)
  for e in range(epochs):
      running_loss = 0
      for images, labels in trainloader:
          images=images.float()
          optimizer.zero_grad()

          log_ps = model(images)
          loss = criterion(log_ps, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
      else:
          test_loss=0
          accuracy=0
          with torch.no_grad():
              model.eval()
          # validation pass here
              for images, labels in testloader:
                  images=images.float()
                  val=model(images)
                  test_loss += criterion(val, labels)
                  val_prob= torch.exp(val)
                  top_p, top_class = val_prob.topk(1, dim=1)
                  
                  equals = top_class == labels.view(*top_class.shape)
                  accuracy += torch.mean(equals.type(torch.FloatTensor))
          model.train()
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(testloader))
          print("Epoch:{}/{}.. ".format(e+1, epochs),
               "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
               "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
               "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
  else:
    with torch.no_grad():
              model.eval()
          # validation pass here
              for images, labels in testloader:
                  images=images.float()
                  val=model(images)
                  val_prob= torch.exp(val)
                  top_p, top_class = val_prob.topk(1, dim=1)
                  student.append(top_class)
              teacher.append(list(student))
              student.clear()
    model.train()
    

Epoch:1/30..  Training Loss: 4.798..  Test Loss: 1.282..  Test Accuracy: 0.594
Epoch:2/30..  Training Loss: 1.516..  Test Loss: 1.011..  Test Accuracy: 0.678
Epoch:3/30..  Training Loss: 1.090..  Test Loss: 0.793..  Test Accuracy: 0.745
Epoch:4/30..  Training Loss: 0.782..  Test Loss: 0.745..  Test Accuracy: 0.765
Epoch:5/30..  Training Loss: 0.551..  Test Loss: 0.690..  Test Accuracy: 0.782
Epoch:6/30..  Training Loss: 0.439..  Test Loss: 0.622..  Test Accuracy: 0.805
Epoch:7/30..  Training Loss: 0.405..  Test Loss: 0.603..  Test Accuracy: 0.817
Epoch:8/30..  Training Loss: 0.284..  Test Loss: 0.636..  Test Accuracy: 0.822
Epoch:9/30..  Training Loss: 0.239..  Test Loss: 0.664..  Test Accuracy: 0.828
Epoch:10/30..  Training Loss: 0.220..  Test Loss: 0.660..  Test Accuracy: 0.833
Epoch:11/30..  Training Loss: 0.219..  Test Loss: 0.671..  Test Accuracy: 0.837
Epoch:12/30..  Training Loss: 0.201..  Test Loss: 0.803..  Test Accuracy: 0.810
Epoch:13/30..  Training Loss: 0.185..  Test Loss:

In [0]:
# teacher_one=teacher[0]
# teacher_two=teacher[1]
# teacher_three=teacher[2]
# teacher_four=teacher[3]
# teacher_five=teacher[4]
# teacher_six=teacher[5]


In [0]:
for i in range(6):
  teacher_tensor=torch.stack(teacher[i])
teacher_tensor=teacher_tensor.view(60000)

NameError: ignored

In [0]:
teacher_list=list()
for i in teacher:
  teacher_list.append(torch.stack(i))


teacher_list=torch.stack(teacher_list)

teacher_list=teacher_list.view(-1,120)

In [0]:
# teacher_test=teacher_test.view(60000)
# teacher_tensor.shape
teacher_list.shape
# len(teacher)

torch.Size([60000, 120])

In [0]:
# teacher_one=torch.stack(teacher_one)
# teacher_two=torch.stack(teacher_two)
# teacher_three=torch.stack(teacher_three)
# teacher_four=torch.stack(teacher_four)
# teacher_five=torch.stack(teacher_five)
# teacher_six=torch.stack(teacher_six)


In [0]:
# teacher_one=teacher_one.view(60000)
# teacher_two=teacher_two.view(60000)
# teacher_three=teacher_three.view(60000)
# teacher_four=teacher_four.view(60000)
# teacher_five=teacher_five.view(60000)
# teacher_six=teacher_six.view(60000)


In [0]:
# teacher_tensor=torch.stack((teacher_one,teacher_two,teacher_three,teacher_four,teacher_five,teacher_six))


In [0]:
# teacher_tensor=teacher_tensor.view(60000)
# teacher_tensor

In [0]:
import numpy as np
new_labels=list()
for an_image in teacher_list:

    label_counts = torch.bincount(an_image, minlength=10)
    label_counts=label_counts.numpy()
    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [0]:
equals = torch.tensor(new_labels) == test_targets
accuracy += torch.mean(equals.type(torch.FloatTensor))

In [0]:
print(accuracy/len(testloader))

tensor(0.7457)


In [0]:
!pip install --upgrade syft

     |████████████████████████████████| 225kB 4.8MB/s 
     |████████████████████████████████| 204kB 49.6MB/s 
     |████████████████████████████████| 256kB 47.3MB/s 
     |████████████████████████████████| 81kB 29.4MB/s 
     |████████████████████████████████| 389kB 52.2MB/s 
     |████████████████████████████████| 1.4MB 47.6MB/s 
     |████████████████████████████████| 450kB 49.3MB/s 
     |████████████████████████████████| 51kB 21.5MB/s 
     |████████████████████████████████| 276kB 39.1MB/s 
     |████████████████████████████████| 122kB 41.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [0]:

from syft.frameworks.torch.differential_privacy import pate
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=teacher_list.view(6,-1).numpy(), indices=np.array(new_labels), noise_eps=0.7, delta=1e-5,moments=20)

print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

In [0]:
from syft.frameworks.torch.differential_privacy import pate
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=teacher_list.view(120,-1).numpy(), indices=np.array(new_labels), noise_eps=0.1 , delta=1e-5,moments=20)

print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 2411.5129254649705
Data Dependent Epsilon: 61.76344555437916


In [0]:
from syft.frameworks.torch.differential_privacy import pate
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=teacher_list[:10,:].view(120,-1).numpy(), indices=np.array(new_labels[:10]), noise_eps=0.1, delta=1e-5,moments=20)

print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 2.5756462732485117
Data Dependent Epsilon: 2.5756462732485117


In [0]:
np.array(new_labels[:10])

array([  8,   8,   1,  26,  49,  49,   3, 104,  23,   7])

In [0]:
test_targets[:10].numpy().shape

(10,)